In [5]:
import pandas as pd
import os
from math import *
import re
from datetime import datetime, date, time, timedelta
import fix_yahoo_finance as yf
from time import sleep

#universe = ['spy','ark'] 
universe = ['spy'] 
#variables for data download
frame = 20 #for limiting the range of optimizations, 1 year
hist_window = frame * 5 #for historical pricing
date_fmt = '%Y-%m-%d'
start_date = datetime.now() - timedelta(hist_window)
start_date = start_date.strftime(date_fmt)
sleep_time = 5

component_path = "./sector_components/"
pricing_path = "./pricing/"

## Script que obtiene los datos de las fuentes:
# 1. Sector components   : Contiene la información de los ETF sectors y sus componentes
# 2. Pricing            : Contiene la informacion historica de precios de cada uno de los componentes

In [2]:
################################################################################################################################
### 1. Sector components:
# de donde se extrae esta info, con que frecuencia se actualiza?

## 1.1 descargar componentes del server:
#### aqui de donde se descargan los datos???????

## 1.2 cargar los sector componentes para cada benchmark, cargar cada ETF sector para cada benchmark
# y cada company para cada ETF sectior y limpiarlos

def load_companies(benchmark,skiprows,columns): ## el input es el nombre del benchmark, si ignora o no el primer row y el nombre de las columnas que son de interes, primero ticker y luego nombre de la compañia
    resultados_benchmark = {}
    resultados_companies = {}
    resultados = pd.DataFrame([])
    ## que ETF SECTOR tiene el benchmark?: calcularlo:
    directory=component_path+benchmark ## directorio donde se buscará los archivos
    file_list=os.listdir(directory)    ## lista de archivos en el directorio
    files = [f for f in file_list if bool(re.search("-"+benchmark+"-", f))] ## lista de archivos .etf sectors
    for key in files: ## loop que recorre todos los etf sectors
        aux = pd.read_csv(directory+"/"+key,skiprows=skiprows,index_col=columns[0])[columns[1]] ## load companies
        aux=aux.to_frame()
        aux.columns=["company_name"]
        aux.index.name = "ticker"
        aux["benchmark"]=benchmark ## agrega columna con nombre del benchmark
        aux["etf_sector"]=re.search("\-([a-z]*?)\.csv",key).groups()[0]  # agrega columna con nonmbre del etf sector
        print("loaded: "+ re.search("\-([a-z]*?)\.csv",key).groups()[0])
        resultados=resultados.append(aux)
        resultados_companies[re.search("\-([a-z]*?)\.csv",key).groups()[0]] = aux.index.tolist()
    resultados_benchmark[benchmark]=resultados.etf_sector.unique().tolist()
    return resultados,resultados_benchmark,resultados_companies


companies_spy,benchmark_spy_etf_map,spy_sector_tickers_map=load_companies(benchmark="spy",skiprows=1,columns=['Symbol', 'Company Name']) ## companies del benchmark spy
print("Companies loaded:", companies_spy.shape)

#companies_ark,benchmark_ark_etf_map1,ark_sector_tickers_map1=load_companies(benchmark="ark",skiprows=0,columns=['ticker','company']) ## companies del benchmark ark
#print("Companies loaded:", companies1.shape)

loaded: xlb
loaded: xle
loaded: xlf
loaded: xli
loaded: xlk
loaded: xlp
loaded: xlre
loaded: xlu
loaded: xlv
loaded: xly
Companies loaded: (505, 4)


In [3]:
### aqui generalizar para mas benchmarks
companies_universe=companies_spy.copy()
sector_etf = benchmark_spy_etf_map.copy()
ticker_map=spy_sector_tickers_map.copy()

In [24]:
################################################################################################################################
# 2. Princing            : Contiene la informacion historica de precios de cada uno de los componentes


## 2.1 descargar componentes del server:

def get_pricing(output_name,ticker_list,start_date):
    try:
        if isinstance(ticker_list, list)==False:
            ticker_list=[ticker_list]
        px = yf.download(ticker_list, start=start_date)['Adj Close']
        if isinstance(px, pd.DataFrame)==False: 
            px=px.to_frame()
            px.columns=ticker_list
        px.sort_index(ascending=True, inplace=True)
        px.to_csv(output_name)
        print(output_name)
        #return px
    except Exception as err:
        print("Error: {0}, waiting to try again in {1}".format(err, sleep_time))
        print(output_name)
        sleep(sleep_time)




In [25]:
### refresh components
etfs=[ [i,j,ticker_map[j]] for i in universe for j in sector_etf[i] ]
for i in etfs:
    get_pricing(output_name=pricing_path+i[0]+"/"+i[1]+"-hold-pricing.csv",
        ticker_list=i[2],start_date=start_date)

[*********************100%***********************]  25 of 25 downloaded
./pricing/spy/xlb-hold-pricing.csv
[*********************100%***********************]  32 of 32 downloaded
./pricing/spy/xle-hold-pricing.csv
[*********************100%***********************]  67 of 67 downloaded
./pricing/spy/xlf-hold-pricing.csv
[*********************100%***********************]  70 of 70 downloaded
./pricing/spy/xli-hold-pricing.csv
[*********************100%***********************]  71 of 71 downloaded
./pricing/spy/xlk-hold-pricing.csv
[*********************100%***********************]  34 of 34 downloaded
./pricing/spy/xlp-hold-pricing.csv
[*********************100%***********************]  33 of 33 downloaded
./pricing/spy/xlre-hold-pricing.csv
[*********************100%***********************]  28 of 28 downloaded
./pricing/spy/xlu-hold-pricing.csv
[*********************100%***********************]  61 of 61 downloaded
./pricing/spy/xlv-hold-pricing.csv
[*********************100%**********

In [26]:
## refresh etf sectors 
sectors=[[i,sector_etf[i]] for i in universe]
for j in sectors:
    get_pricing(output_name=pricing_path+"/"+j[0]+"/"+j[0]+'-sectors.csv', 
        ticker_list=j[1], start_date=start_date)

[*********************100%***********************]  10 of 10 downloaded
./pricing//spy/spy-sectors.csv


In [27]:
# refresh spy index
get_pricing(output_name=pricing_path+'/spy/SPY.csv', ticker_list='SPY', start_date=start_date)


[*********************100%***********************]  1 of 1 downloaded
./pricing//spy/SPY.csv


In [32]:
### consol_px contiene los precios historicos de cada compañia
consol_px=pd.DataFrame()
### crear consol_px 
for i in universe:
    print(i)
    directory=pricing_path+i ## directorio donde se buscará los archivos
    file_list=os.listdir(directory)    ## lista de archivos en el directorio
    files = [f for f in file_list if bool(re.search("-hold-pricing.csv", f))] 
    for f in files:
        aux=pd.read_csv(directory+"/"+f,index_col="Date")
        #aux.index=aux["Date"]
        ccols = set(consol_px.columns.tolist())
        newcols = set(aux.columns.tolist())
        consol_px = consol_px.merge(
            aux[list(newcols.difference(ccols))], 
            left_index=True, 
            right_index=True, 
            how='outer')

consol_px.head()

spy


,MON,NEM,FMC,IFF,MOS,APD,VMC,EMN,SEE,PX,...,TJX,UA,GM,ULTA,WYN,GT,IPG,WHR,MGM,CCL
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-28,118.136887,36.928703,94.335518,153.753082,24.042305,162.297974,121.380882,91.125816,46.823120,154.714920,...,72.349014,11.51,44.066811,217.070007,110.349998,31.684340,19.046230,166.345795,33.026218,66.079857
2017-11-29,117.171326,36.391773,93.047768,153.862595,24.341711,161.681534,124.573532,90.847542,47.001835,153.537445,...,74.470192,12.18,42.977894,223.100006,112.269997,32.013035,19.694529,167.845581,33.423527,65.275085
2017-11-30,117.798439,36.779556,94.235695,154.738632,24.241909,162.099106,125.361725,91.801636,47.868549,152.300583,...,75.237007,11.93,42.271568,221.710007,112.389999,32.242126,19.605324,167.428421,33.890358,65.215477
2017-12-01,118.385742,36.978416,93.766510,154.001968,24.231930,160.707199,120.931908,91.414032,47.559723,150.272156,...,75.037827,11.73,41.977268,212.580002,111.669998,31.993113,19.912586,166.325943,33.791035,65.573151
2017-12-04,118.067207,36.630409,91.310799,153.643585,23.653078,161.462799,124.962639,90.837601,47.938286,151.746475,...,76.680992,11.98,42.232327,221.639999,112.209999,31.943312,20.497375,168.481247,33.989689,66.815063


In [33]:
# #### Limpiar las compañias que tienen valores nulos en los ultimos 3 dias y valores nulos en toda su serie
def clean_nas(df):
    cols = df.count().sort_values()[df.count().sort_values() < 1].index.tolist()  ## precios con toda la serie nula
    aux=df.tail(3)
    cols = list(set(cols+ df.columns[aux.isnull().any()].tolist() ))## precios con al menos las ultimas 3 dias sin precios
    df = df.drop(cols, axis=1) ## quitar empresas que tienen valores nulos en todos sus precios
    df.fillna(method='pad', inplace=True)
    df.fillna(method='bfill', inplace=True)
    cleanmin = lambda x: max(float(x), 1)
    df = df.applymap(cleanmin)
    return df,cols

consol_px,cols=clean_nas(consol_px) ## limpia de aquellos valores que no tienen valores en su serie o que por alguna razon no tienen NAN en los ultimos 3 dias
print("Se borraron las siguientes compañias")
print(cols)
consol_px.head()



Se borraron las siguientes compañias
['SNI', 'BRK.B', 'BF.B']


,MON,NEM,FMC,IFF,MOS,APD,VMC,EMN,SEE,PX,...,TJX,UA,GM,ULTA,WYN,GT,IPG,WHR,MGM,CCL
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-28,118.136887,36.928703,94.335518,153.753082,24.042305,162.297974,121.380882,91.125816,46.823120,154.714920,...,72.349014,11.51,44.066811,217.070007,110.349998,31.684340,19.046230,166.345795,33.026218,66.079857
2017-11-29,117.171326,36.391773,93.047768,153.862595,24.341711,161.681534,124.573532,90.847542,47.001835,153.537445,...,74.470192,12.18,42.977894,223.100006,112.269997,32.013035,19.694529,167.845581,33.423527,65.275085
2017-11-30,117.798439,36.779556,94.235695,154.738632,24.241909,162.099106,125.361725,91.801636,47.868549,152.300583,...,75.237007,11.93,42.271568,221.710007,112.389999,32.242126,19.605324,167.428421,33.890358,65.215477
2017-12-01,118.385742,36.978416,93.766510,154.001968,24.231930,160.707199,120.931908,91.414032,47.559723,150.272156,...,75.037827,11.73,41.977268,212.580002,111.669998,31.993113,19.912586,166.325943,33.791035,65.573151
2017-12-04,118.067207,36.630409,91.310799,153.643585,23.653078,161.462799,124.962639,90.837601,47.938286,151.746475,...,76.680992,11.98,42.232327,221.639999,112.209999,31.943312,20.497375,168.481247,33.989689,66.815063


In [34]:
# Removing tickers for M&A targets and diff class shares of same co.
remove_tickers=['CSRA', 'DPS', 'UAA'] 
print("Se borraron las siguientes compañias")
print(remove_tickers)
consol_px = consol_px.drop(remove_tickers, axis=1)
consol_px.to_pickle("consol_px.pkl")
consol_px.head()

Se borraron las siguientes compañias
['CSRA', 'DPS', 'UAA']


,MON,NEM,FMC,IFF,MOS,APD,VMC,EMN,SEE,PX,...,TJX,UA,GM,ULTA,WYN,GT,IPG,WHR,MGM,CCL
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-28,118.136887,36.928703,94.335518,153.753082,24.042305,162.297974,121.380882,91.125816,46.823120,154.714920,...,72.349014,11.51,44.066811,217.070007,110.349998,31.684340,19.046230,166.345795,33.026218,66.079857
2017-11-29,117.171326,36.391773,93.047768,153.862595,24.341711,161.681534,124.573532,90.847542,47.001835,153.537445,...,74.470192,12.18,42.977894,223.100006,112.269997,32.013035,19.694529,167.845581,33.423527,65.275085
2017-11-30,117.798439,36.779556,94.235695,154.738632,24.241909,162.099106,125.361725,91.801636,47.868549,152.300583,...,75.237007,11.93,42.271568,221.710007,112.389999,32.242126,19.605324,167.428421,33.890358,65.215477
2017-12-01,118.385742,36.978416,93.766510,154.001968,24.231930,160.707199,120.931908,91.414032,47.559723,150.272156,...,75.037827,11.73,41.977268,212.580002,111.669998,31.993113,19.912586,166.325943,33.791035,65.573151
2017-12-04,118.067207,36.630409,91.310799,153.643585,23.653078,161.462799,124.962639,90.837601,47.938286,151.746475,...,76.680992,11.98,42.232327,221.639999,112.209999,31.943312,20.497375,168.481247,33.989689,66.815063
